In [54]:
# Included libraries
import numpy as np
import pandas as pd
from __future__ import annotations
from asyncio import current_task, run
from contextlib import asynccontextmanager
from collections.abc import AsyncGenerator

# SQL
from sqlalchemy import Column, MetaData, select, String, Table, event, text, create_engine, insert, update, delete
from sqlalchemy.engine import Engine
from sqlalchemy.ext.asyncio import create_async_engine, AsyncSession, async_scoped_session
from sqlalchemy.orm import sessionmaker

In [55]:
# Load CSV
df = pd.read_csv("../data/MarketData.csv", header=[3,4,5,6])
df.columns = df.columns.get_level_values(1) # Truncate to 1 header
df.rename(columns={df.columns[0]:"Date"}, inplace=True) # This is a bit inefficient as this name exists in level 0...
df.rename(columns={'LMCADS03 Comdty':'Copper',
                   'LMAHDS03 Comdty':'Aluminium',
                   'LMZSDS03 Comdty':'Zinc',
                   'LMPBDS03 Comdty':'Lead',
                   'LMSNDS03 Comdty':'Tin',
                   'CL1 Comdty':'CL1'},
                    inplace=True)
df.head(5)

,Date,Copper,Aluminium,Zinc,Lead,Tin,CL1
0,01/01/2010,7375.0,2230.0,2560.0,2432.0,16950,79.36
1,04/01/2010,7500.0,2267.0,2574.0,2515.0,17450,81.51
2,05/01/2010,7485.0,2302.0,2575.0,2522.5,17375,81.77
3,06/01/2010,7660.0,2377.0,2718.0,2680.0,17825,83.18
4,07/01/2010,7535.0,2310.0,2607.0,2599.0,17475,82.66


In [56]:
engine = create_async_engine('sqlite+aiosqlite://', echo=False) # SQL engine
meta = MetaData() # Metadata for SQL session - used to generate table object later
maker = sessionmaker(engine, class_=AsyncSession)
scope = async_scoped_session(maker, current_task)
tablename = 'MarketData'

In [57]:
# Generate sessions via contextmanager
@asynccontextmanager
async def get_session():
   async with scope() as session:
      yield session # Yield instead of return to pick up where it left off on every function call

In [58]:
# Asynchronous SQL CRUD operations
async def main():

   # Obtain a session variable for each instance
   async with get_session() as session:

      conn = await session.connection()

      # Asynchronous CSV to SQL
      await conn.run_sync(
         lambda sync_conn: df.to_sql(tablename, con=sync_conn)
      )

      # Define SQL table object from metadata, and add incrementing index as first column
      marketdata = await conn.run_sync(
         lambda conn: Table(tablename, meta, autoload_with=conn, sqlite_autoincrement=True)
      )

      # Create (CRUD)
      async with engine.connect() as conn:
         stmt = insert(marketdata).values(Date="17/02/2024", Copper=8319.0)
         result = await conn.execute(stmt)
         await conn.commit()

      # Update (CRUD)
      async with engine.connect() as conn:
         stmt = update(marketdata).where(marketdata.c.Date=="17/02/2024").values(Copper=8321.0)
         result = await conn.execute(stmt)
         await conn.commit()

      # Delete (CRUD)
      async with engine.connect() as conn:
         stmt = delete(marketdata).where(marketdata.c.Date=="17/02/2024")
         result = await conn.execute(stmt)
         await conn.commit()

      # Read (CRUD)
      async with engine.connect() as conn:
         result = await conn.execute(select(marketdata))
         print(result.fetchall())

      # The Above CRUD operations can also be done using conn.execute(text("")), allowing for raw SQL input format.

      # Permanently store changes to database
      await session.commit()

# Launch asynchronous task
await main()

[(0, '01/01/2010', 7375.0, 2230.0, 2560.0, 2432.0, 16950, 79.36), (1, '04/01/2010', 7500.0, 2267.0, 2574.0, 2515.0, 17450, 81.51), (2, '05/01/2010', 7485.0, 2302.0, 2575.0, 2522.5, 17375, 81.77), (3, '06/01/2010', 7660.0, 2377.0, 2718.0, 2680.0, 17825, 83.18), (4, '07/01/2010', 7535.0, 2310.0, 2607.0, 2599.0, 17475, 82.66), (5, '08/01/2010', 7461.0, 2284.0, 2521.0, 2532.0, 17340, 82.75), (6, '11/01/2010', 7567.5, 2330.0, 2573.0, 2531.0, 17900, 82.52), (7, '12/01/2010', 7455.0, 2281.0, 2475.0, 2432.0, 17900, 80.79), (8, '13/01/2010', 7485.0, 2294.0, 2489.0, 2480.0, 18000, 79.65), (9, '14/01/2010', 7490.0, 2335.0, 2503.0, 2507.0, 18425, 79.39), (10, '15/01/2010', 7430.0, 2306.0, 2470.0, 2435.0, 18100, 78.0), (11, '18/01/2010', 7500.0, 2303.5, 2481.0, 2465.0, 18000, 78.0), (12, '19/01/2010', 7545.0, 2293.5, 2503.0, 2425.0, 17975, 79.02), (13, '20/01/2010', 7375.0, 2267.0, 2444.0, 2295.0, 17775, 77.62), (14, '21/01/2010', 7275.0, 2239.0, 2400.0, 2260.0, 17750, 76.08), (15, '22/01/2010', 73